# Analyzing Arthropoda Taxonomy: Integrating GBIF and NCBI Data

![title](https://wallpapercave.com/wp/wp1870417.jpg)

This Python notebook is designed for the purpose of integrating taxonomic data from two major biological databases, GBIF (Global Biodiversity Information Facility) and NCBI (National Center for Biotechnology Information), to enhance the accuracy and comprehensiveness of ecological and biological research. GBIF primarily focuses on biodiversity data including species distribution and ecological information, whereas NCBI provides a broader range of data including genomic and taxonomic details. 

Combining these sources enables researchers to cross-validate species identifications and improve the richness of ecological datasets with genetic information. A key biological task performed in this notebook is the construction of a taxonomic tree, which helps in visualizing and understanding the evolutionary relationships and classification hierarchy among different species within a chosen taxon (in this case, the Arthropda pyhlum).

## 1. Importing libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [7]:
import taxonmatch as txm

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 2. Downloading and processing samples

The initial steps involve downloading the most recent taxonomic data from GBIF and NCBI to ensure the analysis is based on the latest available information. 

In [8]:
gbif_dataset = txm.download_gbif_taxonomy()

NameError: name 'txm' is not defined

In [9]:
ncbi_dataset = txm.download_ncbi_taxonomy()

NameError: name 'txm' is not defined

## 2.1 Checking Inconsistencies in nomenclature

In [ ]:
df_inconsistencies = txm.get_inconsistencies(gbif_dataset, ncbi_dataset)

In [ ]:
df = df_inconsistencies [df_inconsistencies ['canonicalName'].apply(lambda x: len(x.split()) == 2)]

In [ ]:
df.sample(10)

## 3.a Training the classifier model

If required, the notebook outlines steps to train a machine learning classifier to distinguish between correct and incorrect taxonomic matches. This involves generating positive and negative examples, preparing the training dataset, and comparing different models.

In [8]:
positive_matches = txm.generate_positive_set(gbif_dataset, ncbi_dataset, 50000)

In [ ]:
negative_matches = txm.generate_negative_set(gbif_dataset, ncbi_dataset, 50000)

 samples 13608 out of 16667

In [ ]:
full_training_set = txm.prepare_data(positive_matches, negative_matches)

In [ ]:
full_training_set.to_csv("training_set.txt", index = False)

In [12]:
X_train, X_test, y_train, y_test = txm.generate_training_test(full_training_set)

In [13]:
txm.compare_models(X_train, X_test, y_train, y_test)

,model,accuracy,mae,precision,recall,f1,roc,run_time,tp,fp,tn,fn
0,XGBClassifier,0.979772,0.020228,0.977194,0.982667,0.979923,0.979758,0.0,14515,344,14740,260
1,RandomForestClassifier,0.979169,0.020831,0.975652,0.983067,0.979345,0.979150,0.11,14491,368,14746,254
2,KNeighborsClassifier,0.972404,0.027596,0.968783,0.976533,0.972643,0.972384,0.02,14387,472,14648,352
3,DecisionTreeClassifier,0.969657,0.030343,0.967866,0.971867,0.969862,0.969647,0.01,14375,484,14578,422
4,GradientBoostingClassifier,0.965806,0.034194,0.957099,0.975667,0.966294,0.965759,0.13,14203,656,14635,365
5,MLPClassifier,0.960447,0.039553,0.959317,0.962067,0.960690,0.960440,0.65,14247,612,14431,569
6,AdaBoostClassifier,0.938645,0.061355,0.919577,0.962000,0.940310,0.938534,0.03,13597,1262,14430,570
7,SVC,0.930741,0.069259,0.923556,0.939933,0.931673,0.930698,0.89,13692,1167,14099,901
8,Perceptron,0.885797,0.114203,0.921026,0.845133,0.881449,0.885990,0.0,13772,1087,12677,2323
9,DummyClassifier,0.498175,0.501825,0.500530,0.503400,0.501961,0.498150,0.0,7324,7535,7551,7449


In [ ]:
model = txm.XGBClassifier(learning_rate=0.1,n_estimators=500, max_depth=9, n_jobs=-1, colsample_bytree = 1, subsample = 0.8)

In [ ]:
model.fit(X_train, y_train, verbose=False)

In [ ]:
#with open('./files/model/xgb_model.pkl', 'wb') as file:
#    pickle.dump(model, file)

## 3.b Load a pre-trained model

Alternatively, it provides the option to load a pre-trained model, simplifying the process for routine analyses.

In [8]:
from taxonmatch.loader import load_xgb_model 
model = load_xgb_model()

## 4. Match NCBI with GBIF dataset 

In this section, the focus is on comparing and aligning the taxonomic data from NCBI and GBIF datasets. It specifically targets the taxon "Apidae" to narrow down the analysis to a specific family of bees. Using a pre-trained machine learning model, the notebook matches records from both datasets, categorizing them as exact matches, unmatched, or potentially mislabeled due to typographical errors

In [9]:
gbif_arthropda, ncbi_arthropoda = txm.select_taxonomic_clade("Arthropoda", gbif_dataset, ncbi_dataset)

In [10]:
matched_df, unmatched_df, possible_typos_df = txm.match_dataset(gbif_arthropda, ncbi_arthropoda, model, tree_generation = True) 

## 5. Generate the taxonomical tree 

In the last section, the notebook constructs a taxonomic tree from the matched and unmatched data between the GBIF and NCBI datasets, focusing on the Apidae family. This visual representation helps to illustrate the evolutionary relationships and classification hierarchy among the species. The tree is then converted into a dataframe for further analysis and saved in textual format for documentation and review purposes.

In [11]:
tree = txm.generate_taxonomic_tree(matched_df, unmatched_df)

In [12]:
df_from_tree = txm.convert_tree_to_dataframe(tree, gbif_arthropda, ncbi_arthropoda, "taxonomic_tree_df.txt")

In [107]:
new_tree = txm.reroot_tree(tree, root_name="Cicadetta")

In [108]:
txm.print_tree(new_tree)

cicadetta (NCBI ID: 139461, GBIF ID: 4407744)
├── cicadetta abscondita (NCBI ID: 2593298, GBIF ID: 7844511)
├── cicadetta adbominalis (NCBI ID: None, GBIF ID: 12219534)
├── cicadetta afghanistanica (NCBI ID: None, GBIF ID: 7518756)
├── cicadetta albipennis (NCBI ID: None, GBIF ID: 4482651)
├── cicadetta anapaistica (NCBI ID: 1740310, GBIF ID: 8414980)
│   ├── cicadetta anapaistica anapaistica (NCBI ID: None, GBIF ID: 11192347)
│   └── cicadetta anapaistica lucana (NCBI ID: 1889248, GBIF ID: 11198679)
├── cicadetta brevipennis (NCBI ID: 1740311, GBIF ID: 7614613)
│   ├── cicadetta brevipennis brevipennis (NCBI ID: None, GBIF ID: 9428064)
│   ├── cicadetta brevipennis hippolaidica (NCBI ID: None, GBIF ID: 11135606)
│   └── cicadetta brevipennis litoralis (NCBI ID: None, GBIF ID: 10243165)
├── cicadetta brevipennis sensu lato 439bg (NCBI ID: 1889244, GBIF ID: None)
├── cicadetta brevipennis sensu lato 440bg (NCBI ID: 1889245, GBIF ID: None)
├── cicadetta calliope (NCBI ID: 139462, GBIF ID

In [110]:
txm.save_tree(new_tree, "cicadetta_tree.txt", output_format='txt')

The tree is saved as TXT in the file: cicadetta_tree.txt.


In [104]:
def reroot_tree(tree, root_name=None):
    """
    Reroots the tree to start from the specified root node.

    Args:
    tree (AnyNode or similar tree node): The current tree.
    root_name (str, optional): The name of the node to use as the new root, case-insensitive.

    Returns:
    AnyNode: The new root of the tree if root_name is specified and found, otherwise the original tree.
    """
    if root_name is not None:
        root_name = root_name.lower()  # Convert root_name to lowercase for case-insensitive search
        new_root = find_node_by_name(tree, root_name)  # Use the txm prefix for find_node_by_name
        if new_root is None:
            print("Root node not found.")
            return None  # Return None if the root node is not found
        return new_root
    return tree  # Return the original tree if no root_name is specified

In [105]:
from anytree import RenderTree

def print_tree(tree, root_name=None):
    """
    Prints the structure of a tree sorted alphabetically starting from a specified root node.

    Args:
    tree (AnyNode or a similar tree node): The root node to start from.
    root_name (str, optional): The name of the node to use as the root for printing, case-insensitive.

    Each node of the tree may have attributes 'ncbi_id' and 'gbif_taxon_id'.
    The nodes are printed with their names, and IDs are included if available.
    """
    def sort_children(node):
        # Ensure that node.children is a list of nodes and sort it
        if isinstance(node, list):
            return sorted(node, key=lambda child: child.name.lower())
        else:
            return sorted(node.children, key=lambda child: child.name.lower())

    if root_name is not None:
        root_name = root_name.lower()  # Convert root_name to lowercase for case-insensitive search
        tree = find_node_by_name(tree, root_name)  # Use the txm prefix for find_node_by_name
        if tree is None:
            print("Root node not found.")
            return

    # Print the tree from the new root node, sorted alphabetically
    for pre, fill, node in RenderTree(tree, childiter=sort_children):
        ncbi_id = getattr(node, 'ncbi_id', None)
        gbif_id = getattr(node, 'gbif_taxon_id', None)
        id_info = f" (NCBI ID: {ncbi_id}, GBIF ID: {gbif_id})" if ncbi_id or gbif_id else ""
        print(f"{pre}{node.name}{id_info}")

In [106]:
import json
from anytree.exporter import JsonExporter
from anytree import RenderTree

def tree_to_newick(node):
    """
    Recursively converts an AnyNode tree to Newick format.
    """
    if not node.children:
        return node.name
    children_newick = ",".join([tree_to_newick(child) for child in node.children])
    return f"({children_newick}){node.name}"

def save_tree(tree, path, output_format='txt'):
    """
    Saves a tree structure to a file with identification details for each node in various formats.
    Args:
    tree (AnyNode or similar tree node): The root of the tree to be saved.
    path (str): Path to the file where the tree will be saved.
    output_format (str): Format to save the tree. Supported formats: 'txt', 'newick', 'json'.
    """
    def alphabetical_sort(node):
        return node.name.lower()  # Sort nodes alphabetically by name

    def sort_children(node):
        # Sort the children of the current node, if it has any
        if hasattr(node, 'children') and node.children:
            node.children = sorted(node.children, key=lambda child: child.name.lower())
            # Recursively sort the children of each child node
            for child in node.children:
                sort_children(child)

    # Sort the tree before saving
    sort_children(tree)

    if output_format == 'txt':
        with open(path, 'w') as f:
            # Print the tree with IDs to the file
            for pre, fill, node in RenderTree(tree):
                ncbi_id = getattr(node, 'ncbi_id', None)
                gbif_id = getattr(node, 'gbif_taxon_id', None)
                id_info = f" (NCBI ID: {ncbi_id}, GBIF ID: {gbif_id})" if ncbi_id or gbif_id else ""
                f.write(f"{pre}{node.name}{id_info}\n")
        print(f"The tree is saved as TXT in the file: {path}.")

    elif output_format == 'newick':
        newick_representation = tree_to_newick(tree) + ';'
        with open(path, 'w') as f:
            f.write(newick_representation)
        print(f"The tree is saved as Newick in the file: {path}.")

    elif output_format == 'json':
        exporter = JsonExporter(indent=2, sort_keys=True)
        json_data = exporter.export(tree)
        with open(path, 'w') as f:
            f.write(json_data)
        print(f"The tree is saved as JSON in the file: {path}.")

    else:
        raise ValueError(f"Unsupported format: {output_format}. Supported formats are 'txt', 'newick', and 'json'.")

